# Importing libraries


In [ ]:
import pandas as pd 
import json
import urllib
import requests, re
from datetime import datetime
from youtube_api import YouTubeDataAPI
from youtube_api import youtube_api_utils, P
from runtimestamp.runtimestamp import runtimestamp
runtimestamp()

Updated 2021-05-03 23:54:27.748883
By danilapadarouski
Using Python 3.7.6
On Darwin-20.2.0-x86_64-i386-64bit


# Initializing api key

In [ ]:
api_key = 'Enter your key'
yt = YouTubeDataAPI(api_key)

# Functions that create dataset from channel id

In [ ]:
def preprocessing_channels(yt, channel):
    """
    Getting dataset from channels' id
    """
    channel_meta = yt.get_channel_metadata(channel)
    playlist_id = channel_meta['playlist_id_uploads']
    video_ids = yt.get_videos_from_playlist_id(playlist_id)
    df = pd.DataFrame(video_ids)
    video_id = df['video_id'].tolist()
    video_meta = yt.get_video_metadata(video_id, parser = P.parse_video_metadata)
    df = pd.DataFrame(video_meta)
    df['region'] = channel_meta['country']
    df['subscriptions'] = channel_meta['subscription_count']
    return df

In [ ]:
def feature_engeneering(df): 
    
    #Adding weekday of uploading and date of publishing to dataset
    weekday = list()
    publish_date = list()

    for date in df['video_publish_date']:
        weekday_date = datetime.utcfromtimestamp(date).strftime('%A')
        weekday.append(weekday_date)
        transformed_date = datetime.utcfromtimestamp(date).strftime('%Y-%m-%d %H:%M:%S')
        publish_date.append(transformed_date)
    
    df['publish_date'] = publish_date
    df['weekday'] = weekday
    
    #Include all videos that were published in 2021
    df = df.where(df['publish_date'] > '2021-01-01 00:00:01')
    df.dropna(inplace = True)
    
    
    #Adding durations of videos to dataset
    durations = list()
    
    for video_id in df['video_id']:
        search_url = f'https://www.googleapis.com/youtube/v3/videos?id={video_id}&key={api_key}&part=contentDetails'
        req = urllib.request.Request(search_url)
        response = urllib.request.urlopen(req).read().decode('utf-8')
        data = json.loads(response)
        all_data = data['items']
        duration = all_data[0]['contentDetails']['duration']
        durations.append(duration)
    
    df['duration'] = durations
    
    #Deleting unused columns
    del df['channel_id']
    del df['video_thumbnail']
    del df['collection_date']
    
    #Like ratio
    df['LR'] = df['video_like_count'].astype(int) / df['video_view_count'].astype(int)
    
    #Dislike ratio
    df['DR'] = df['video_dislike_count'].astype(int) / df['video_view_count'].astype(int)
    
    #View ratio
    df['VR'] = df['video_view_count'].astype(int) / df['subscriptions'].astype(int)
    
    return df

In [ ]:
def final_processing_channels(yt, channel):
    df = preprocessing_channels(yt = yt, channel = channel)
    df = feature_engeneering(df = df)
    return df

# Getting list of most popular channels by country

In [ ]:
r = requests.get("https://www.noxinfluencer.com/youtube-channel-rank/top-250-by-all-youtuber-sorted-by-subs-weekly")
a = re.compile('/youtube/(channel)/([^ "]*)')
list_of_channels = re.findall(a, r.text)

# Getting dataframe of videos of all popular channels in the country

In [ ]:
df_channels = pd.DataFrame()
for channel in list_of_channels:
    df_tmp = final_processing(yt = yt, channel = channel)
    df_channels = pd.concat([df_tmp, df_channels], ignore_index = True)

In [ ]:
df_channels

,channel_title,video_title,video_description,video_category,video_view_count,video_comment_count,video_like_count,video_dislike_count,video_tags,region,subscriptions,publish_date,weekday,duration,LR,DR,VR
0,ВЕРТЕИЧ,ЗВОНЮ ПОДПИСЧИКАМ,Мой сервер в SAMP: https://arizona-rp.com/​​​​...,20,131145,1278,16439,110,вертеич,BY,561000,2021-04-27 14:40:01,Tuesday,PT5M26S,0.125350,0.000839,0.233770
1,ВЕРТЕИЧ,БЕСКОНЕЧНЫЕ ДЕНЬГИ,Скачать игру: https://rodina-rp.com/howtostart...,20,172225,1013,14779,102,деньги|бесконечные деньги|деньги гта|вертеич|с...,BY,561000,2021-04-21 07:39:49,Wednesday,PT6M8S,0.085812,0.000592,0.306996
2,ВЕРТЕИЧ,МЕНТЫ - БЛИЗНЕЦЫ,Филлини: https://www.youtube.com/channel/UCn1R...,20,139361,1339,16194,427,вертеич|гта|филлини|самп|samp|гта самп,BY,561000,2021-04-19 10:20:44,Monday,PT5M48S,0.116202,0.003064,0.248415
3,ВЕРТЕИЧ,ИЗДЕВАЮСЬ НАД СЕРВЕРОМ в SAMP,Мой сервер в SAMP: https://arizona-rp.com/​​​​...,20,196320,2240,33222,290,samp|самп|гта самп|сервер самп|сервер|вертеич|...,BY,561000,2021-04-14 14:40:47,Wednesday,PT5M12S,0.169224,0.001477,0.349947
4,ВЕРТЕИЧ,КУПИЛ САМУЮ ДОРОГУЮ АДМИНКУ на НУБО РП в SAMP,💸 Пиши сюда и зарабатывай на поиске каналов - ...,20,234312,1073,17997,284,samp|самп|гта|нубо рп|вертеич|админка|купил|са...,BY,561000,2021-04-13 13:33:23,Tuesday,PT6M20S,0.076808,0.001212,0.417668
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28369,A4,Экстремальные ПРЯТКИ от ЗОМБИ Челлендж !,Покупай Мерч А4 «Ламба&Гелик» на официальном с...,24,18566562,232870,1650067,49696,Экстремальные ПРЯТКИ от ЗОМБИ Челлендж !|прятк...,BY,30300000,2021-01-19 08:00:26,Tuesday,PT23M33S,0.088873,0.002677,0.612758
28370,A4,15 Способов Пронести СЛАДОСТИ в КИНОТЕАТР ! **...,Покупай Мерч А4 «Ламба&Гелик» на официальном с...,24,20977000,274526,1623023,53326,15 Способов Пронести СЛАДОСТИ в КИНОТЕАТР !|15...,BY,30300000,2021-01-15 08:00:35,Friday,PT24M41S,0.077372,0.002542,0.692310
28371,A4,ХОЛОДНАЯ vs ГОРЯЧАЯ Тюрьма Челлендж !,Покупай Мерч А4 «Ламба&Гелик» на официальном с...,24,20167994,85594,1021446,43470,ХОЛОДНАЯ vs ГОРЯЧАЯ Тюрьма Челлендж !|холодная...,BY,30300000,2021-01-12 08:00:16,Tuesday,PT20M59S,0.050647,0.002155,0.665610
28372,A4,Самая Сильная ЖЕНЩИНА в МИРЕ vs Непробиваемый ...,Распродажа на Joom с 8 по 10 января - https://...,24,22118808,123400,1286255,58173,Самая Сильная ЖЕНЩИНА в МИРЕ vs Непробиваемый ...,BY,30300000,2021-01-07 04:00:10,Thursday,PT25M9S,0.058152,0.002630,0.729994


# Saving dataframe

In [ ]:
df_channels.to_csv("data_belarus.csv")